In [ ]:
!pip3 install googlesearch-python
!pip3 install tld
!pip3 install SPF2IP

In [17]:
import pandas as pd
from tld import get_tld, is_tld
import re
from googlesearch import search
from SPF2IP import SPF2IP
import requests

In [112]:
df = pd.read_csv("../data/malicious_phish.csv")
origin = pd.read_csv("../data/dataset_cybersecurity_michelle.csv")

origin.columns.tolist()

['qty_dot_url',
 'qty_hyphen_url',
 'qty_underline_url',
 'qty_slash_url',
 'qty_questionmark_url',
 'qty_equal_url',
 'qty_at_url',
 'qty_and_url',
 'qty_exclamation_url',
 'qty_space_url',
 'qty_tilde_url',
 'qty_comma_url',
 'qty_plus_url',
 'qty_asterisk_url',
 'qty_hashtag_url',
 'qty_dollar_url',
 'qty_percent_url',
 'qty_tld_url',
 'length_url',
 'qty_dot_domain',
 'qty_hyphen_domain',
 'qty_underline_domain',
 'qty_slash_domain',
 'qty_questionmark_domain',
 'qty_equal_domain',
 'qty_at_domain',
 'qty_and_domain',
 'qty_exclamation_domain',
 'qty_space_domain',
 'qty_tilde_domain',
 'qty_comma_domain',
 'qty_plus_domain',
 'qty_asterisk_domain',
 'qty_hashtag_domain',
 'qty_dollar_domain',
 'qty_percent_domain',
 'qty_vowels_domain',
 'domain_length',
 'domain_in_ip',
 'server_client_domain',
 'qty_dot_directory',
 'qty_hyphen_directory',
 'qty_underline_directory',
 'qty_slash_directory',
 'qty_questionmark_directory',
 'qty_equal_directory',
 'qty_at_directory',
 'qty_and_dir

In [77]:
#df_phish = df.copy()
df_phish = df.loc[(df['type'] == 'benign') | (df['type'] == 'phishing')]
df_phish['type'].value_counts()


type
benign      428103
phishing     94111
Name: count, dtype: int64

In [78]:
# add url len + get domain
def process_tld(url):
    try:
        res = get_tld(url, as_object=True, fail_silently=False, fix_protocol=True)
        domain = res.parsed_url.netloc
    except :
        domain = None
    return domain

df_phish.loc[:,'domain'] = df_phish.loc[:,'url'].apply(lambda i: process_tld(i))
df_phish.loc[:,'length_url'] = df_phish.loc[:,'url'].apply(lambda x: len(str(x)))
df_phish.loc[:,'domain_length'] = df_phish.loc[:,'domain'].apply(lambda x: len(str(x)))
df_phish.head()

/var/folders/m9/xh9774s15gz65l_hkjkrkv640000gn/T/ipykernel_683/2178137929.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_phish.loc[:,'domain'] = df_phish.loc[:,'url'].apply(lambda i: process_tld(i))
/var/folders/m9/xh9774s15gz65l_hkjkrkv640000gn/T/ipykernel_683/2178137929.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_phish.loc[:,'length_url'] = df_phish.loc[:,'url'].apply(lambda x: len(str(x)))
/var/folders/m9/xh9774s15gz65l_hkjkrkv640000gn/T/ipykernel_683/2178137929.py:12: SettingWithCop

,url,type,domain,length_url,domain_length
0,br-icloud.com.br,phishing,br-icloud.com.br,16,16
1,mp3raid.com/music/krizz_kaliko.html,benign,mp3raid.com,35,11
2,bopsecrets.org/rexroth/cr/1.htm,benign,bopsecrets.org,31,14
5,http://buzzfil.net/m/show-art/ils-etaient-loin...,benign,buzzfil.net,118,11
6,espn.go.com/nba/player/_/id/3457/brandon-rush,benign,espn.go.com,45,11


### Reformat url dataset

In [ ]:
vowels=['a','e','i','o','u']
features = {'at':'@', 'questionmark':'?', 'underline':'_', 'hyphen':'-', 'equal':'=', 'dot':'.', 
            'hashtag':'#', 'percent':'%', 'plus':'+', 'dollar':'$', 'exclamation':'!', 'asterisk':'*', 
            'comma':',', 'slash':'/', 'space':' ', 'tilde':'~','and':'&'}

# for qtr associated with url
for key, value in features.items():
    df_phish.loc[:, "qty_" + key + "_url"] = df_phish.loc[:, 'url'].apply(lambda i: i.count(value))

# for qtr associated with domain
for key, value in features.items():
    df_phish.loc[:, "qty_" + key + "_domain"] = df_phish.loc[:, 'domain'].apply(lambda i: 0 if i is None else i.count(value))

# Add a new column named 'vowel_count' with vowel counts for domain
df_phish.loc[:,"qty_vowels_domain"] = df_phish.loc[:,'domain'].apply(lambda x: 0 if x is None else sum(char.lower() in vowels for char in x))

# check if url is shortened
def shortened(url):
    match = re.search(
                      'bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
    if match:
        return 1
    else:
        return 0
df_phish.loc[:,'url_shortened'] = df_phish.loc[:,'url'].apply(lambda x: shortened(x))

# check if google index is available for url & domain
def google_index(url):
    site = search(url, 3)
    return 1 if site else 0

df_phish['url_google_index'] = df_phish['url'].apply(lambda i: google_index(i))
df_phish['domain_google_index'] = df_phish['domain'].apply(lambda i: google_index(i))

# check if email in url
regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b'
def check_email(url):
    if(re.fullmatch(regex, url)):
        return 1
    else:
        return 0
df_phish['email_in_url'] = df_phish['url'].apply(lambda i: check_email(i))

# check if domain has spf record
def check_spf(dom):
    try:
        spf_records = SPF2IP().query(dom)
        if spf_records:
            return 1
        else:
            return 0
    except:
        return 0

df_phish['domain_spf'] = df_phish['domain'].apply(check_spf)

# count top level domain in url
def count_tlds(url):
    # Regular expression pattern to extract TLDs from a URL
    tld_pattern = r'\.([a-zA-Z]{2,})$'
    
    # Find all matches of the TLD pattern in the URL
    tlds = re.findall(tld_pattern, url)

    #if 
    
    # Return the count of unique TLDs
    return len(set(tlds))

df_phish['qty_tld_url']=df_phish['url'].apply(count_tlds)

In [116]:
#df_phish['qty_tld_url']=df_phish['url'].apply(count_tlds)
df_phish.head()

,url,type,domain,length_url,domain_length,qty_at_url,qty_questionmark_url,qty_underline_url,qty_hyphen_url,qty_equal_url,...,qty_space_domain,qty_tilde_domain,qty_and_domain,qty_vowels_domain,url_shortened,url_google_index,domain_google_index,email_in_url,domain_spf,qty_tld_url
0,br-icloud.com.br,phishing,br-icloud.com.br,16,16,0,0,0,1,0,...,0,0,0,4,0,1,1,0,0,1
1,mp3raid.com/music/krizz_kaliko.html,benign,mp3raid.com,35,11,0,0,1,0,0,...,0,0,0,3,0,1,1,0,0,1
2,bopsecrets.org/rexroth/cr/1.htm,benign,bopsecrets.org,31,14,0,0,0,0,0,...,0,0,0,4,0,1,1,0,0,1
5,http://buzzfil.net/m/show-art/ils-etaient-loin...,benign,buzzfil.net,118,11,0,0,0,16,0,...,0,0,0,3,0,1,1,0,0,1
6,espn.go.com/nba/player/_/id/3457/brandon-rush,benign,espn.go.com,45,11,0,0,1,1,0,...,0,0,0,3,0,1,1,0,0,0


In [ ]:
# Use of IP or not in domain
# def ip_in_domain(url):
#     match = re.search(
#         '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
#         '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # Ipv4
#         '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)' # IPv4 in hexadecimal
#         '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', url)  # Ipv6
#     if match:
#         # print match.group()
#         return 1
#     else:
#         # print 'No matching pattern found'
#         return 0
# df_phish.loc[:,'ip_in_domain'] = df_phish.loc[:,'url'].apply(lambda i: ip_in_domain(i))

In [117]:
df_phish.columns
#df_phish.rename(columns={'qtr_tilde_url': 'qty_tilde_url'}, inplace=True)

Index(['url', 'type', 'domain', 'length_url', 'domain_length', 'qty_at_url',
       'qty_questionmark_url', 'qty_underline_url', 'qty_hyphen_url',
       'qty_equal_url', 'qty_dot_url', 'qty_hashtag_url', 'qty_percent_url',
       'qty_plus_url', 'qty_dollar_url', 'qty_exclamation_url',
       'qty_asterisk_url', 'qty_comma_url', 'qty_slash_url', 'qty_space_url',
       'qty_tilde_url', 'qty_and_url', 'qty_at_domain',
       'qty_questionmark_domain', 'qty_underline_domain', 'qty_hyphen_domain',
       'qty_equal_domain', 'qty_dot_domain', 'qty_hashtag_domain',
       'qty_percent_domain', 'qty_plus_domain', 'qty_dollar_domain',
       'qty_exclamation_domain', 'qty_asterisk_domain', 'qty_comma_domain',
       'qty_slash_domain', 'qty_space_domain', 'qty_tilde_domain',
       'qty_and_domain', 'qty_vowels_domain', 'url_shortened',
       'url_google_index', 'domain_google_index', 'email_in_url', 'domain_spf',
       'qty_tld_url'],
      dtype='object')

In [119]:
# Check columns present in orginal df but not in df_phish
missing_columns = origin.columns.difference(df_phish.columns)

# Remove unwanted columns from missing_column
columns_to_add = missing_columns.delete([5,])
#Add missing columns to df_phish with default value of 0
for column in missing_columns:
     df_phish.loc[:,column] = 0

type(missing_columns)
missing_columns[5]

'phishing'

In [ ]:
# obtain ttl for hostname of url
# import dns.resolver

# def get_ttl(url):
#     try:
#         result = dns.resolver.resolve(url, 'A')
#         if result.response.answer:
#             return int(result.response.answer[0].ttl)
#     except (dns.resolver.NoAnswer, dns.resolver.NXDOMAIN,
#             dns.exception.Timeout, dns.resolver.NoNameservers):
#         return 0
#     return 0

# df_phish.loc[:,'ttl_hostname'] = df_phish.loc[:,'domain'].apply(lambda i: get_ttl(i))

# import requests
# def check_ssl_certificate(url):
#     try:
#         response = requests.head(url)
#         # Check if the response status code is 200 (OK) or 301 (Moved Permanently)
#         if response.status_code == 200 or response.status_code == 301:
#             return 1  # SSL certificate is available
#     except requests.exceptions.SSLError:
#         pass  # SSL certificate is not available or there is an SSL error
#     except requests.exceptions.RequestException:
#         pass  # Handle other request exceptions if needed
#     return 0  # SSL certificate is not available

# df_phish.loc[:,'tls_ssl_certificate	'] = df_phish.loc[:,'url'].apply(check_ssl_certificate)